# 사용자 정의 Document Loader 생성 방법

## 개요

LLM 기반 애플리케이션은 종종 PDF와 같은 데이터베이스나 파일에서 데이터를 추출하고, 이를 LLM이 사용할 수 있는 형식으로 변환하는 작업을 포함합니다. 

LangChain에서는 일반적으로 추출된 텍스트(`page_content`)와 메타데이터(예: 저자 이름이나 출판 날짜 등의 문서에 대한 세부 정보를 포함하는 사전)를 함께 묶은 Document 객체를 생성하는 것을 포함합니다.

`Document` 객체는 종종 LLM에 공급되는 프롬프트로 포맷되어, LLM이 `Document`의 정보를 사용하여 원하는 응답(예: 문서 요약)을 생성할 수 있게 합니다. 

Document 로딩의 주요 추상화는 다음과 같습니다

| 구성요소        | 설명                                      |
|----------------|------------------------------------------|
| Document       | `text`와 `metadata`를 포함합니다.         |
| BaseLoader     | 원시 데이터를 `Documents`로 변환하는 데 사용합니다. |

`BaseLoader`에서 서브클래싱하여 표준 문서 Loader를 생성합니다.

## 표준 문서 로더

문서 로더는 문서를 로드하는 표준 인터페이스를 제공하는 `BaseLoader`에서 서브클래싱하여 구현할 수 있습니다.

### 인터페이스

| 메서드 이름 | 설명 |
|-------------|-------------|
| lazy_load   | 문서를 하나씩 **게으르게** 로드하는 데 사용됩니다. 생산 코드에 사용하세요. |
| alazy_load  | `lazy_load`의 비동기 변형입니다. |
| load        | 모든 문서를 메모리에 **즉시** 로드하는 데 사용됩니다. 프로토타이핑이나 대화형 작업에 사용하세요. |
| aload       | 모든 문서를 메모리에 **즉시** 로드하는 데 사용됩니다. 프로토타이핑이나 대화형 작업에 사용하세요. **2024-04에 LangChain에 추가되었습니다.** |

* `load` 메서드는 프로토타이핑 작업을 위한 편리한 메서드로, 단순히 `list(self.lazy_load())`를 호출합니다.

* `alazy_load`는 기본적으로 `lazy_load`에 위임하는 기본 구현을 가지고 있습니다. 비동기를 사용하는 경우, 기본 구현을 오버라이딩하고 네이티브 비동기 구현을 제공하는 것이 좋습니다.

**중요**

문서 로더를 구현할 때 `lazy_load` 또는 `alazy_load` 메서드를 통해 매개변수를 제공하지 **마십시오**.

모든 구성은 초기화자(__init__)를 통해 전달되어야 합니다. 이는 LangChain에서 문서 로더가 인스턴스화되면 모든 문서를 로드하는 데 필요한 정보를 갖추도록 하기 위한 설계 선택이었습니다.


### 구현

파일을 로드하고 파일의 각 줄에서 문서를 생성하는 표준 문서 로더의 예를 만들어 보겠습니다.

In [ ]:
# 설치
# !pip install langchain olefile llama-index-core llama-parse llama-index-readers-file

In [1]:
from typing import Iterator

from langchain_core.document_loaders import BaseLoader
from langchain_core.documents import Document


class CustomDocumentLoader(BaseLoader):
    """파일을 한 줄씩 읽어오는 문서 로더의 예시입니다."""

    def __init__(self, file_path: str) -> None:
        """로더를 파일 경로와 함께 초기화합니다.

        Args:
            file_path: 로드할 파일의 경로입니다.
        """
        self.file_path = file_path

    def lazy_load(self) -> Iterator[Document]:  # <-- 인자를 받지 않습니다
        """파일을 한 줄씩 읽어오는 지연 로더입니다.

        지연 로드 메소드를 구현할 때는, 문서를 하나씩 생성하여 반환하는 제너레이터를 사용해야 합니다.
        """
        with open(self.file_path, encoding="utf-8") as f:
            line_number = 0
            for line in f:
                yield Document(
                    page_content=line,
                    metadata={"line_number": line_number, "source": self.file_path},
                )
                line_number += 1

이 코드에서 'lazy'라는 용어는 '지연 로딩(lazy loading)' 또는 '지연 평가(lazy evaluation)'의 개념을 나타냅니다. 

1. 메모리 효율성: 'lazy'는 모든 데이터를 한 번에 메모리에 로드하지 않고, 필요할 때만 로드한다는 의미입니다.

2. 성능 최적화: 대량의 데이터를 다룰 때, 필요한 부분만 처리하므로 초기 로딩 시간과 메모리 사용량을 줄일 수 있습니다.
3. 제너레이터 사용: `lazy_load` 메서드는 제너레이터를 반환하여 데이터를 하나씩 순차적으로 생성합니다.
4. 비동기 지원: `alazy_load` 메서드는 비동기 방식으로 lazy loading을 구현합니다.
5. 유연성: 사용자가 데이터를 어떻게 처리할지 선택할 수 있게 해줍니다. 모든 데이터가 필요하면 `load()` 메서드를, 부분적으로 필요하면 `lazy_load()` 메서드를 사용할 수 있습니다.
6. 추상화: `BaseLoader` 클래스는 lazy loading을 기본 동작으로 정의하고, 하위 클래스들이 이를 구현하도록 유도합니다.

In [123]:
from pathlib import Path
from typing import Any, Dict, List, Optional
import olefile
import zlib
import struct
import re
import unicodedata
from langchain.schema import Document
from langchain.document_loaders.base import BaseLoader


class HWPReader(BaseLoader):
    """HWP 파일 읽기 클래스. HWP 파일의 내용을 읽습니다."""

    def __init__(self, file_path: str, *args: Any, **kwargs: Any) -> None:
        super().__init__(*args, **kwargs)
        self.file_path = file_path
        self.extra_info = None
        self._initialize_constants()

    def _initialize_constants(self) -> None:
        """상수 초기화 메서드"""
        self.FILE_HEADER_SECTION = "FileHeader"
        self.HWP_SUMMARY_SECTION = "\x05HwpSummaryInformation"
        self.SECTION_NAME_LENGTH = len("Section")
        self.BODYTEXT_SECTION = "BodyText"
        self.HWP_TEXT_TAGS = [67]

    def lazy_load(self) -> List[Document]:
        """HWP 파일에서 데이터를 로드하고 표를 추출합니다.

        Returns:
            List[Document]: 추출된 문서 리스트
        """
        load_file = olefile.OleFileIO(self.file_path)
        file_dir = load_file.listdir()

        if not self._is_valid_hwp(file_dir):
            raise ValueError("유효하지 않은 HWP 파일입니다.")

        result_text = self._extract_text(load_file, file_dir)
        return [self._create_document(text=result_text, extra_info=self.extra_info)]

    def _is_valid_hwp(self, dirs: List[List[str]]) -> bool:
        """HWP 파일의 유효성을 검사합니다."""
        return [self.FILE_HEADER_SECTION] in dirs and [self.HWP_SUMMARY_SECTION] in dirs

    def _get_body_sections(self, dirs: List[List[str]]) -> List[str]:
        """본문 섹션 목록을 반환합니다."""
        section_numbers = [
            int(d[1][self.SECTION_NAME_LENGTH :])
            for d in dirs
            if d[0] == self.BODYTEXT_SECTION
        ]
        return [
            f"{self.BODYTEXT_SECTION}/Section{num}" for num in sorted(section_numbers)
        ]

    def _create_document(
        self, text: str, extra_info: Optional[Dict] = None
    ) -> Document:
        """문서 객체를 생성합니다."""
        return Document(page_content=text, metadata=extra_info or {})

    def _extract_text(
        self, load_file: olefile.OleFileIO, file_dir: List[List[str]]
    ) -> str:
        """모든 섹션에서 텍스트를 추출합니다."""
        sections = self._get_body_sections(file_dir)
        return "\n".join(
            self._get_text_from_section(load_file, section) for section in sections
        )

    def _is_compressed(self, load_file: olefile.OleFileIO) -> bool:
        """파일이 압축되었는지 확인합니다."""
        with load_file.openstream(self.FILE_HEADER_SECTION) as header:
            header_data = header.read()
            return bool(header_data[36] & 1)

    def _get_text_from_section(self, load_file: olefile.OleFileIO, section: str) -> str:
        """특정 섹션에서 텍스트를 추출합니다."""
        with load_file.openstream(section) as bodytext:
            data = bodytext.read()

        unpacked_data = (
            zlib.decompress(data, -15) if self._is_compressed(load_file) else data
        )

        text = []
        i = 0
        while i < len(unpacked_data):
            header, rec_type, rec_len = self._parse_record_header(
                unpacked_data[i : i + 4]
            )
            if rec_type in self.HWP_TEXT_TAGS:
                rec_data = unpacked_data[i + 4 : i + 4 + rec_len]
                text.append(rec_data.decode("utf-16"))
            i += 4 + rec_len

        text = "\n".join(text)
        text = self.remove_chinese_characters(text)
        text = self.remove_control_characters(text)
        return text

    @staticmethod
    def remove_chinese_characters(s: str):
        """중국어 문자를 제거합니다."""
        return re.sub(r"[\u4e00-\u9fff]+", "", s)

    @staticmethod
    def remove_control_characters(s):
        """깨지는 문자 제거"""
        return "".join(ch for ch in s if unicodedata.category(ch)[0] != "C")

    @staticmethod
    def _parse_record_header(header_bytes: bytes) -> tuple:
        """레코드 헤더를 파싱합니다."""
        header = struct.unpack_from("<I", header_bytes)[0]
        rec_type = header & 0x3FF
        rec_len = (header >> 20) & 0xFFF
        return header, rec_type, rec_len

In [ ]:
## 17조(무슨 법) 어쩌구 저쩌구
  ### 1항 
    3조 2항에 따라 어쩌구 저쩌구
  ### 2항
    3조 3항에 따라 어쩌구 저쩌구

In [124]:
loader = HWPReader("data/디지털 정부혁신 추진계획.hwp")
print(loader.load()[0].page_content)

디지털 정부혁신 추진계획2019. 10. 29.      관계부처 합동순    서Ⅰ. 개요ȃ 1Ⅱ. 디지털 정부혁신 추진계획ㆬȃ 2  1. 우선 추진과제ȃ 2     ① 선제적·통합적 대국민 서비스 혁신     ② 공공부문 마이데이터 활성화     ③ 시민참여를 위한 플랫폼 고도화     ④ 현장중심 협업을 지원하는 스마트 업무환경 구현     ⑤ 클라우드와 디지털서비스 이용 활성화     ⑥ 개방형 데이터·서비스 생태계 구축  2. 중장기 범정부 디지털 전환 로드맵 수립ᲈȃ 4Ⅲ. 추진체계 및 일정ȃ 4 [붙임] 디지털 정부혁신 우선 추진과제(상세)ᬜȃ 8Ⅰ. 개 요□ 추진 배경 ○ 우리나라는 국가적 초고속 정보통신망 투자와 적극적인 공공정보화 사업 추진에 힘입어 세계 최고수준의 전자정부를 구축‧운영     * UN전자정부평가에서 2010‧12‧14년 1위, 16‧18년 3위, UN공공행정상 13회 수상 ○ 그러나, 인공지능‧클라우드 중심의 디지털 전환(Digital Transformation) 시대가 도래함에 따라 기존 전자정부의 한계 표출   - 축적된 행정데이터에도 불구하고 기관간 연계‧활용 미흡, 부처 단위로 단절된 서비스, 신기술 활용을 위한 제도‧기반 부족   - 디지털 전환을 위한 컨트롤타워가 없고, 구체적 전략도 부재 ○ 이에, ‘19.3월부터 공공부문 ICT 활용현황 및 문제점 검토에 착수하여 공공분야 디지털 전환을 위한 추진계획 마련     * 관계부처 협의 21회(행안,과기정통,기재,복지,권익위,국정원 등), 민간전문가 의견청취 10회□ 문제점 진단 및 평가 ○ (서비스) 국민과 최종 이용자 관점에서 서비스 혁신 미흡   - 자격이 있어도 자신이 받을 수 있는 공공서비스를 파악하기 어려워 사각지대가 발생하고, 온라인 신청 가능한 서비스도 제한적 ○ (데이터) 기관별로 축적·보유한 데이터의 연계와 활용 부족   - A기관에서 서류를 발급받아 B기관에 제출하는 관행(연간 증명서 9.5억건‘18년 발급) 등 데이터가 국민편익 향상에 제대로 활용

In [128]:
print(loader.load()[0].page_content[-500:])

 플랫폼에서 이용 가능   -개인정보보호 및 보안체계를 갖춘 적격 민간사업자 대상 개방     ※안전성 확보를 위한 사전 평가 등 공공서비스 개방체계 마련 ○ 공공기관에서 직접 개발·운영하고 있는 불편하고, 활용도가 낮은 공공 앱들은 대폭 감축     ※18년말 기준 총 771개의 공공앱 운영 중 : 총 구축·운영비 989억원, 평균 누적다운로드 24.3만건, 이용자수 9.3만명, 이용자평점 2.8(이용자수가 500명 미만인 앱도 202개) ○ 민간에서 많이 활용하는 소셜 로그인, 간편결제, 화상회의 등의 검증된 온라인서비스를 공공 시스템에서도 도입·활용 확대     ※ 공공 영역에서 자연어처리, 이미지·음성 인식, 인공지능, 데이터분석 등 민간의 첨단 솔루션을 전략적으로 활용하여 디지털 신산업을 육성하는 방안 적극 추진참고4주요 서비스별 변화 모습□ 선제적 맞춤형서비스□ 생애주기 원스톱서비스□ 디지털 고지 활성화□ 전자증명서 발급·유통□ 현장중심의 스마트 업무환경□ 공공서비스 개방


In [125]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = text_splitter.split_documents(loader.load())

In [126]:
split_docs[:10]

[Document(page_content='디지털 정부혁신 추진계획2019. 10. 29.      관계부처 합동순    서Ⅰ. 개요ȃ 1Ⅱ. 디지털 정부혁신 추진계획ㆬȃ 2  1. 우선 추진과제ȃ 2     ① 선제적·통합적 대국민 서비스 혁신     ② 공공부문 마이데이터 활성화     ③ 시민참여를 위한 플랫폼 고도화     ④ 현장중심 협업을 지원하는 스마트 업무환경 구현     ⑤ 클라우드와 디지털서비스 이용 활성화     ⑥ 개방형 데이터·서비스 생태계 구축  2. 중장기 범정부 디지털 전환 로드맵 수립ᲈȃ 4Ⅲ. 추진체계 및 일정ȃ 4 [붙임] 디지털 정부혁신 우선 추진과제(상세)ᬜȃ 8Ⅰ. 개 요□ 추진 배경 ○ 우리나라는 국가적 초고속 정보통신망 투자와 적극적인 공공정보화 사업 추진에 힘입어 세계 최고수준의 전자정부를 구축‧운영     * UN전자정부평가에서 2010‧12‧14년 1위, 16‧18년 3위, UN공공행정상 13회 수상 ○ 그러나, 인공지능‧클라우드 중심의 디지털 전환(Digital'),
 Document(page_content='13회 수상 ○ 그러나, 인공지능‧클라우드 중심의 디지털 전환(Digital Transformation) 시대가 도래함에 따라 기존 전자정부의 한계 표출   - 축적된 행정데이터에도 불구하고 기관간 연계‧활용 미흡, 부처 단위로 단절된 서비스, 신기술 활용을 위한 제도‧기반 부족   - 디지털 전환을 위한 컨트롤타워가 없고, 구체적 전략도 부재 ○ 이에, ‘19.3월부터 공공부문 ICT 활용현황 및 문제점 검토에 착수하여 공공분야 디지털 전환을 위한 추진계획 마련     * 관계부처 협의 21회(행안,과기정통,기재,복지,권익위,국정원 등), 민간전문가 의견청취 10회□ 문제점 진단 및 평가 ○ (서비스) 국민과 최종 이용자 관점에서 서비스 혁신 미흡   - 자격이 있어도 자신이 받을 수 있는 공공서비스를 파악하기 어려워 사각지대가 발생하고, 온라인 신청 가능한 서비스도 제한적 ○ (데이터) 기관별로 축적·

## 방법 2) HWP -> PDF 변환 후 텍스트 추출

In [56]:
from langchain_community.document_loaders import PyMuPDFLoader

# PyMuPDF 로더 인스턴스 생성
loader = PyMuPDFLoader("data/디지털 정부혁신 추진계획.pdf")
# 문서 로드
data = loader.load()
# 첫 번째 페이지 데이터 접근
data[0]

Document(metadata={'source': 'data/디지털 정부혁신 추진계획.pdf', 'file_path': 'data/디지털 정부혁신 추진계획.pdf', 'page': 0, 'total_pages': 21, 'format': 'PDF 1.4', 'title': '', 'author': 'user', 'subject': '', 'keywords': '', 'creator': 'Hwp 2018 10.0.0.13764', 'producer': 'Hancom PDF 1.3.0.542', 'creationDate': "D:20240725173128+09'00'", 'modDate': "D:20240725173128+09'00'", 'trapped': ''}, page_content='디지털 정부혁신 추진계획\n2019. 10. 29.\n관계부처 합동\n')

In [58]:
data[0].page_content

'디지털 정부혁신 추진계획\n2019. 10. 29.\n관계부처 합동\n'

In [129]:
print(data[-1].page_content)

□ 전자증명서 발급·유통
□ 현장중심의 스마트 업무환경
□ 공공서비스 개방



In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader

# PyMuPDF 로더 인스턴스 생성
loader = PyMuPDFLoader("data/디지털 정부혁신 추진계획.pdf")
# 문서 로드
data = loader.load()
# 첫 번째 페이지 데이터 접근
data[0]

## LlamaParser

- 링크: https://cloud.llamaindex.ai

In [ ]:
# 설치
# !pip install llama-index-core llama-parse llama-index-readers-file

In [130]:
# 환경 변수에서 LLAMA_CLOUD_API_KEY 불러오기
from dotenv import load_dotenv

load_dotenv()

True

In [131]:
import nest_asyncio

nest_asyncio.apply()

In [132]:
# 필요한 라이브러리 임포트
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader
import os

# LlamaParse 설정
parser = LlamaParse(
    # api_key="llx-...",  # API 키 (환경 변수 LLAMA_CLOUD_API_KEY에 저장 가능)
    result_type="markdown",  # 결과 타입: "markdown" 또는 "text"
    num_workers=4,  # 여러 파일 처리 시 API 호출 분할 수
    verbose=True,
    language="ko",  # 언어 설정 (기본값: 'en')
    skip_diagonal_text=True,
    use_vendor_multimodal_model=True,
    vendor_multimodal_model_name="openai-gpt4o",
    vendor_multimodal_api_key=os.environ.get("OPENAI_API_KEY"),
    # parsing_instruction="Be sure to parse tables in markdown format. Images, Graphs, Diagrams should be interpreted as text with detailed descriptions.",
)

# SimpleDirectoryReader를 사용하여 파일 파싱
file_extractor = {".pdf": parser}

documents = SimpleDirectoryReader(
    input_files=["data/디지털 정부혁신 추진계획.pdf"], file_extractor=file_extractor
).load_data()

# 결과 출력
print(documents)

Started parsing the file under job_id cac11eca-8a4e-41ec-aacf-6d4145ffb937
[Document(id_='78dd1ed1-082b-423b-ae5a-99bd416ee2ad', embedding=None, metadata={'file_path': 'data/디지털 정부혁신 추진계획.pdf', 'file_name': '디지털 정부혁신 추진계획.pdf', 'file_type': 'application/pdf', 'file_size': 1113217, 'creation_date': '2024-07-25', 'last_modified_date': '2024-07-25'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='# 디지털 정부혁신 추진계획\n\n2019. 10. 29.\n\n관계부처 합동', mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), Document(id_='40240350-5ef1-406b-940d-3c1226942b8d', embedding=None, metadata={'file_path': 'data/디지털 정부혁신 추진계획.pdf', 'file_name': '디지털 정부혁신 추진계획.pdf'

In [79]:
documents[0].text

'# 디지털 정부혁신 추진계획\n\n2019. 10. 29.\n\n관계부처 합동'

In [133]:
print(documents[-1].text)

# 전자증명서 발급 · 유통

## 지금까지는
- 복잡한 대출신청
  - 내라는 서류가 이렇게나 많은데, 남들은 대출을 어떻게 받은 걸까?

## 앞으로는
- 증빙서류 없는 전자증명서로 간편한 대출이용
  - 필요한 서류는 내 전자지갑에 쏙! 제출은 클릭 없이 클릭만!

# 현장중심의 스마트 업무환경

## 지금까지는
- 책상 앞의 공무원
  - 아무리 보안이 중요하다지만 컴퓨터 두 대로 오락가락 작업이나 트렁크에서 업무 효율도 떨어지네...

## 앞으로는
- 1인 1노트북
  - 시간 활용의 자유와 업무 효율의 극대화
  - 노트북만 대로 언제 어디서나 업무를 처리할 수 있게 되다니... 일할 맛 난다!

# 공공서비스 개방

## 지금까지는
- 개별적인 앱을 각각 설치
  - 자주 쓰지도 않는 앱을 각각 깔아야 하나? 디자인도 촌스럽고, 오류도 왜 이렇게 많은지...

## 앞으로는
- 공공서비스 개방으로 민간 앱에서도 공공서비스 이용 가능
  - 이제, 내가 자주 쓰는 앱에서도 공공서비스 이용 가능!


In [81]:
# LlamaParse 설정
parser = LlamaParse(
    # api_key="llx-...",  # API 키 (환경 변수 LLAMA_CLOUD_API_KEY에 저장 가능)
    result_type="markdown",  # 결과 타입: "markdown" 또는 "text"
    num_workers=4,  # 여러 파일 처리 시 API 호출 분할 수
    verbose=True,
    language="ko",  # 언어 설정 (기본값: 'en')
    skip_diagonal_text=True,
    use_vendor_multimodal_model=True,
    vendor_multimodal_model_name="openai-gpt4o",
    vendor_multimodal_api_key=os.environ.get("OPENAI_API_KEY"),
    parsing_instruction="""This document is related to the Digital Government Transformation Initiative.
Be sure to parse tables and should be interpreted as text with detailed informations. 
Images, Graphs, Diagrams should be interpreted as text with detailed descriptions.""",
)

# SimpleDirectoryReader를 사용하여 파일 파싱
file_extractor = {".pdf": parser}

documents_with_prompts = SimpleDirectoryReader(
    input_files=["data/디지털 정부혁신 추진계획.pdf"], file_extractor=file_extractor
).load_data()

Started parsing the file under job_id cac11eca-9091-41a9-97f0-8cce22dee743


In [82]:
documents[0].text

'# 디지털 정부혁신 추진계획\n\n2019. 10. 29.\n\n관계부처 합동'

In [87]:
print(documents[-1].text[:200])

# 전자증명서 발급 · 유통

## 지금까지는
- 복잡한 대출신청
  - 내라는 서류가 이렇게나 많은데, 남들은 대출을 어떻게 받은 걸까?

## 앞으로는
- 증빙서류 없는 전자증명서로 간편한 대출이용
  - 필요한 서류는 내 전자지갑에 쏙! 제출은 클릭 없이 클릭만!

# 현장중심의 스마트 업무환경

## 지금까지는
- 책상 앞의 공무원
  - 아무리 보


In [88]:
print(documents_with_prompts[-1].text[:200])

# 전자증명서 발급·유통

**지금까지는**
- 복잡한 대출신청
- 은행서류심사
- 각종 서류 준비
- 내라는 서류가 이렇게나 많은데, 남들은 대출을 어떻게 받은 걸까?

**앞으로는**
- 증빙서류 없는 전자증명서로 간편한 대출이용
- 필요한 서류는 내 전자지갑에 속! 제출은 클릭 없이 클릭만!

# 현장중심의 스마트 업무환경

**지금까지는**
- 책상


In [97]:
print(documents_with_prompts[7].text)

# 참고 2 주요 과제별 · 연도별 추진일정

## 추진 과제

| 추진일정 | 2019 | 2020 | 2021 | 2022 |
| --- | --- | --- | --- | --- |
| 상 | 하 | 상 | 하 | 상 | 하 | 상 | 하 |

### 1. 선제적·통합적 대국민 서비스 혁신
1. 대국민 편의서비스 맞춤형 안내
2. 생애주기 서비스 대폭 확대
3. 사각지대 예방을 위한 선제적 서비스 제공
4. 기존 한계를 뛰어넘는 공공서비스 혁신사업 추진
5. 디지털서비스 표준 마련·적용

| 보조금 | 보조지원 | 공고 | 공고 | 공고 | 공고 |
| --- | --- | --- | --- | --- | --- |
| 2개 | 4개 | 7개 | 10개 | 행안부, 각 부처, 지자체 |

| 마스터플랜 시스템 구축 | 시범 | 본사업 |
| --- | --- | --- |
| 21년 예비사업 | 행안부, 과기정통부, 각 부처 |

| 사례 | 표준 마련 | 단계적 적용 | 확산 |
| --- | --- | --- | --- |
| 행안부, 과기정통부, 각 부처 |

### 2. 공공부문 마이데이터 활성화
1. 민원서류에 대한 자기정보 활용
2. 스마트폰을 통한 전자증명서 발급·유통
3. 공공부문 자기정보 다운로드 서비스 개시
4. 모바일 신분증 도입
5. 디지털 고지·수납 활성화

| 법제도 | 적용 | 확산 |
| --- | --- | --- |
| 행안부, 각 부처, 지자체 |

| 시범 | 10종 | 300종 |
| --- | --- | --- |
| 행안부 |

| 포털 구축 | 시범 | 확대 |
| --- | --- | --- |
| 행안부 |

| 1종 | 3종 |
| --- | --- |
| 행안부, 교육부, 여가 인사처 등 |

| BPR/ISP | 고지·납부 | 고지·납부 |
| --- | --- | --- |
| 자동 추진 | 행안부, 각 부처, 지자체 |

### 3. 시민 참여를 위한 플랫폼 고도화
1. 국민의 소리 청취·분석 시스템

In [106]:
documents = [doc.to_langchain_format() for doc in documents_with_prompts]
print(documents[7].page_content)

# 참고 2 주요 과제별 · 연도별 추진일정

## 추진 과제

| 추진일정 | 2019 | 2020 | 2021 | 2022 |
| --- | --- | --- | --- | --- |
| 상 | 하 | 상 | 하 | 상 | 하 | 상 | 하 |

### 1. 선제적·통합적 대국민 서비스 혁신
1. 대국민 편의서비스 맞춤형 안내
2. 생애주기 서비스 대폭 확대
3. 사각지대 예방을 위한 선제적 서비스 제공
4. 기존 한계를 뛰어넘는 공공서비스 혁신사업 추진
5. 디지털서비스 표준 마련·적용

| 보조금 | 보조지원 | 공고 | 공고 | 공고 | 공고 |
| --- | --- | --- | --- | --- | --- |
| 2개 | 4개 | 7개 | 10개 | 행안부, 각 부처, 지자체 |

| 마스터플랜 시스템 구축 | 시범 | 본사업 |
| --- | --- | --- |
| 21년 예비사업 | 행안부, 과기정통부, 각 부처 |

| 사례 | 표준 마련 | 단계적 적용 | 확산 |
| --- | --- | --- | --- |
| 행안부, 과기정통부, 각 부처 |

### 2. 공공부문 마이데이터 활성화
1. 민원서류에 대한 자기정보 활용
2. 스마트폰을 통한 전자증명서 발급·유통
3. 공공부문 자기정보 다운로드 서비스 개시
4. 모바일 신분증 도입
5. 디지털 고지·수납 활성화

| 법제도 | 적용 | 확산 |
| --- | --- | --- |
| 행안부, 각 부처, 지자체 |

| 시범 | 10종 | 300종 |
| --- | --- | --- |
| 행안부 |

| 포털 구축 | 시범 | 확대 |
| --- | --- | --- |
| 행안부 |

| 1종 | 3종 |
| --- | --- |
| 행안부, 교육부, 여가 인사처 등 |

| BPR/ISP | 고지·납부 | 고지·납부 |
| --- | --- | --- |
| 자동 추진 | 행안부, 각 부처, 지자체 |

### 3. 시민 참여를 위한 플랫폼 고도화
1. 국민의 소리 청취·분석 시스템

## LlamaParse 로더 구현 

In [112]:
from typing import Iterator

from langchain_core.document_loaders import BaseLoader
from langchain_core.documents import Document


class LlamaParseLoader(BaseLoader):
    """파일을 한 줄씩 읽어오는 문서 로더의 예시입니다."""

    def __init__(self, file_paths: List[str], parsing_instructions="") -> None:
        """로더를 파일 경로와 함께 초기화합니다.
        Args:
            file_paths: 로드할 파일의 경로입니다.
        """
        # LlamaParse 설정
        parser = LlamaParse(
            # api_key="llx-...",  # API 키 (환경 변수 LLAMA_CLOUD_API_KEY에 저장 가능)
            result_type="markdown",  # 결과 타입: "markdown" 또는 "text"
            num_workers=4,  # 여러 파일 처리 시 API 호출 분할 수
            verbose=True,
            language="ko",  # 언어 설정 (기본값: 'en')
            invalidate_cache=True,
            skip_diagonal_text=True,
            use_vendor_multimodal_model=True,
            vendor_multimodal_model_name="openai-gpt4o",
            vendor_multimodal_api_key=os.environ.get("OPENAI_API_KEY"),
            parsing_instruction=parsing_instructions,
        )

        file_extractor = {".pdf": parser}

        self.document_reader = SimpleDirectoryReader(
            input_files=file_paths,
            file_extractor=file_extractor,
        )

    def lazy_load(self) -> List[Document]:  # <-- 인자를 받지 않습니다
        """파일을 한 줄씩 읽어오는 지연 로더입니다.

        지연 로드 메소드를 구현할 때는, 문서를 하나씩 생성하여 반환하는 제너레이터를 사용해야 합니다.
        """
        documents = self.document_reader.load_data()
        langchain_documents = [doc.to_langchain_format() for doc in documents]
        return langchain_documents

In [114]:
parsing_instructions = """This document is related to the Digital Government Transformation Initiative.
Be sure to parse tables and should be interpreted as text with detailed informations. 
Images, Graphs, Diagrams should be interpreted as text with detailed descriptions."""

loader = LlamaParseLoader(
    file_paths=["data/디지털 정부혁신 추진계획.pdf"],
    parsing_instructions=parsing_instructions,
)

In [116]:
parsed_documents = loader.load()

Started parsing the file under job_id e521a34b-23e6-457a-b9d9-bb4235c445b7


In [119]:
print(parsed_documents[7].page_content)


# 참고 2 주요 과제별 · 연도별 추진일정

## 추진 과제

| 추진일정 | 2019 | 2020 | 2021 | 2022 | 관계 부처 |
| --- | --- | --- | --- | --- | --- |
| 상 | 하 | 상 | 하 | 상 | 하 | 상 | 하 | |

### 1. 선제적·통합적 대국민 서비스 혁신
1. 대국민 편의서비스 맞춤형 안내
2. 생애주기 서비스 대폭 확대
3. 사각지대 예방을 위한 선제적 서비스 제공
4. 기존 한계를 뛰어넘는 공공서비스 혁신사업 추진
5. 디지털서비스 표준 마련·적용

| 보조금\* | 복지·지자체 | 공고·고지·안내 | 고도화 | 행안부, 각 부처, 지자체 |
| --- | --- | --- | --- | --- |
| 2개 | 4개 | 7개 | 10개 | 행안부, 복지부, 교육부 등 |
| 마스터플랜 시스템 구축 | 시범 | 본사업 | 복지부, 부처별 지자체 |
| 21년 예비사업 | 행안·과기정통부, 관계부처 |
| 사례 | 표준 마련 | 단계적 적용 | 확산 | 행안·과기정통부, 관계부처 |

### 2. 공공부문 마이데이터 활성화
1. 민원서류에 대한 자기정보 활용
2. 스마트폰을 통한 전자증명서 발급·유통
3. 공공부문 자기정보 다운로드 서비스 개시
4. 모바일 신분증 도입
5. 디지털 고지·수납 활성화

| 법개정 | 적용 | 확산 | 행안부, 각 부처, 지자체 |
| --- | --- | --- | --- |
| 시범 | 10종 | 300종 | 행안부 |
| 포털 구축 | 행안부 |
| 시범 | 확대 | 행안부 |
| 1종 | 3종 | 행안부, 교육, 여가, 인사처 등 |
| BPR/ISP | 고지·납부 고도화 | 자동 추진 | 행안부, 각 부처, 지자체 |

### 3. 시민 참여를 위한 플랫폼 고도화
1. 국민의 소리 청취·분석 시스템 개선
2. 도전.한국 플랫폼 운영
3. 디지털 취약계층을 위한 지원

| 전자데이터 개방 | BPR/ISP | 1차 | 2차 | 행안부, 각 부처, 지자체

## lazy_load() 에 충실한 방법

In [120]:
from typing import Iterator, List
import os
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader
from langchain_core.document_loaders import BaseLoader
from langchain_core.documents import Document


class LlamaParseLoader(BaseLoader):
    """파일을 한 줄씩 읽어오는 문서 로더의 예시입니다."""

    def __init__(self, file_paths: List[str], parsing_instructions="") -> None:
        """로더를 파일 경로와 함께 초기화합니다.
        Args:
            file_paths: 로드할 파일의 경로입니다.
            parsing_instructions: 파싱 지침입니다.
        """
        # LlamaParse 설정
        parser = LlamaParse(
            result_type="markdown",
            num_workers=4,
            verbose=True,
            language="ko",
            invalidate_cache=True,
            skip_diagonal_text=True,
            use_vendor_multimodal_model=True,
            vendor_multimodal_model_name="openai-gpt4o",
            vendor_multimodal_api_key=os.environ.get("OPENAI_API_KEY"),
            parsing_instruction=parsing_instructions,
        )

        file_extractor = {".pdf": parser}

        self.document_reader = SimpleDirectoryReader(
            input_files=file_paths,
            file_extractor=file_extractor,
        )

    def lazy_load(self) -> Iterator[Document]:
        """파일을 한 줄씩 읽어오는 지연 로더입니다.

        문서를 하나씩 생성하여 반환하는 제너레이터를 사용합니다.
        """
        for doc in self.document_reader.load_data():
            yield doc.to_langchain_format()

In [121]:
parsing_instructions = """This document is related to the Digital Government Transformation Initiative.
Be sure to parse tables and should be interpreted as text with detailed informations. 
Images, Graphs, Diagrams should be interpreted as text with detailed descriptions."""

loader = LlamaParseLoader(
    file_paths=["data/디지털 정부혁신 추진계획.pdf"],
    parsing_instructions=parsing_instructions,
)

In [122]:
loader.load()

Started parsing the file under job_id 148233ca-979b-41e4-bd1a-31256a97497e
.

[Document(metadata={'file_path': 'data/디지털 정부혁신 추진계획.pdf', 'file_name': '디지털 정부혁신 추진계획.pdf', 'file_type': 'application/pdf', 'file_size': 1113217, 'creation_date': '2024-07-25', 'last_modified_date': '2024-07-25'}, page_content='\n# 디지털 정부혁신 추진계획\n'),
 Document(metadata={'file_path': 'data/디지털 정부혁신 추진계획.pdf', 'file_name': '디지털 정부혁신 추진계획.pdf', 'file_type': 'application/pdf', 'file_size': 1113217, 'creation_date': '2024-07-25', 'last_modified_date': '2024-07-25'}, page_content='\n2019. 10. 29.\n'),
 Document(metadata={'file_path': 'data/디지털 정부혁신 추진계획.pdf', 'file_name': '디지털 정부혁신 추진계획.pdf', 'file_type': 'application/pdf', 'file_size': 1113217, 'creation_date': '2024-07-25', 'last_modified_date': '2024-07-25'}, page_content='\n관계부처 합동\n'),
 Document(metadata={'file_path': 'data/디지털 정부혁신 추진계획.pdf', 'file_name': '디지털 정부혁신 추진계획.pdf', 'file_type': 'application/pdf', 'file_size': 1113217, 'creation_date': '2024-07-25', 'last_modified_date': '2024-07-25'}, page_content='# 순서\n\n## I. 개요 ........